## Setup

In [2]:
from utils import *

## Zones

In [3]:
bgs = gpd.read_parquet('data/zones.parquet', filters=[
    ('level', '==', 'BG')], columns=['geoid', 'geometry']).disp()

106,044 rows x 2 cols; Memory: 7.8 MiB; <Geographic 2D CRS: EPSG:4326>


,geoid,geometry
,<object>,<geometry>
0,280330705233,"POLYGON ((-89.972174 34.977195, -89.970229 34...."


## Job counts

Download LODES data (https://lehd.ces.census.gov/data) for states which contain at least one CBG of the 50 largest cities.

In [4]:
imp_states = (pd.read_parquet('data/states.parquet', columns=['geoid', 'code'])
              .merge(bgs.geoid.str[:2], on='geoid').drop_duplicates().code)
Arr(imp_states)

array(['AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'IL', 'IN',
       'KS', 'KY', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'NV', 'NH', 'NJ',
       'NY', 'NC', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'TN', 'TX', 'UT',
       'VA', 'WA', 'WI'], dtype=object)

Download the job count data for 2021 using the Workplace Area Characteristics (WAC) table of the LODES v8 dataset. For Arizona (AR) and Mississippi (MS), the latest data is available for 2018 (for which the dataset version is v7).

In [5]:
# jobs = []
# for abbr in tqdm(imp_states):
#     # download data for 2021 except for AL & MS (outdated data)
#     year = 2018 if abbr in ['AR', 'MS'] else 2021
#     version = 7 if year <= 2020 else 8
#     cols = D(w_geocode='geoid', C000='All', CE01='Low wage', CR01='White',
#          CD01='< high school', CD02='high school')
#     url = (f'https://lehd.ces.census.gov/data/lodes/LODES{version}/' + 
#            '{0}/wac/{0}_wac_S000_JT00_{1}.csv.gz'.format(abbr.lower(), year))
#     df = pd.read_csv(url, usecols=list(cols)).rename(columns=cols)
#     df['Low edu'] = df.pop('< high school') + df.pop('high school')
#     df['POC'] = df['All'] - df.pop('White')
#     df.geoid = df.geoid.astype(str).str.zfill(15)
#     for level, nchar in [('BG', 12), ('Tract', 11), ('County', 5)]:
#         df2 = df.assign(geoid=df.geoid.str[:nchar])
#         df2 = df2.groupby('geoid').sum().reset_index()
#         df2 = df2.melt('geoid', var_name='kind', value_name='njobs')
#         jobs.append(df2.query('njobs > 0').assign(level=level))
# cols = D(geoid=CAT, level=CAT, kind=CAT, njobs=I32)
# jobs = pd.concat(jobs).reset_index(drop=1).astype(cols)[list(cols)].disp()
# jobs.to_parquet('data/jobs.parquet') # 50s
jobs = pd.read_parquet('data/jobs.parquet').disp()

1,157,081 rows x 4 cols; Memory: 38.3 MiB


,geoid,level,kind,njobs
,<category>,<category>,<category>,<int32>
0,040019426001,BG,All,4


## POIs
Process the Points of Interest (POI) data from the SafeGraph dataset of Nov 2020 obtained previously as part of the open research data sharing contract during Covid.

In [6]:
poi_cats = Pdf(sum([[(k, x) for x in v] for k, v in {
    'Education': [
        'Colleges, Universities, and Professional Schools',
        'Elementary and Secondary Schools',
        'Other Schools and Instruction',
    ],
    'Groceries': [
        'Department Stores',
        'General Merchandise Stores, Including Warehouse Clubs and Supercenters',
        'Grocery Stores',
    ],
    'Medical': [
        'General Medical and Surgical Hospitals',
        'Medical and Diagnostic Laboratories',
        'Nursing Care Facilities (Skilled Nursing Facilities)',
        'Offices of Dentists',
        'Offices of Other Health Practitioners',
        'Offices of Physicians',
        'Other Ambulatory Health Care Services',
        'Outpatient Care Centers',
        'Specialty (except Psychiatric and Substance Abuse) Hospitals',
    ],
    'Social Support': [
        'Child Day Care Services',
        'Community Food and Housing, and Emergency and Other Relief Services',
        'Continuing Care Retirement Communities and Assisted Living Facilities for the Elderly',
        'Home Health Care Services',
    ]
}.items()], []), columns=['kind', 'top_category']).disp()

19 rows x 2 cols; Memory: 0.0 MiB


,kind,top_category
,<object>,<object>
0,Education,"Colleges, Universities, and Professional Schools"


In [7]:
# pois = pd.read_parquet(
#     '/home/umni2/a/umnilab/data/safegraph/pois/us/2020-11-06.parquet',
#     columns=['top_category', 'lon', 'lat'])
# pois = (pdf2gdf(pois, crs=CRS_DEG)
#         .sjoin(bgs, predicate='within')
#         .groupby(['geoid', 'top_category']).size()
#         .rename('npois').reset_index()
#         .merge(poi_cats, 'left', on='top_category'))
# bg_pois = pd.concat([
#     pois.groupby('geoid').npois.sum().reset_index().assign(kind='All'),
#     pois.groupby(['kind', 'geoid']).npois.sum().reset_index()
# ]).reset_index(drop=1).assign(level='BG')
# pois = [bg_pois]
# for level, nchar in [('Tract', 11), ('County', 5)]:
#     df = bg_pois.assign(geoid=bg_pois.geoid.str[:nchar])
#     df = df.groupby(['geoid', 'kind']).npois.sum().reset_index()
#     pois.append(df.assign(level=level))
# pois = pd.concat(pois).reset_index(drop=1)
# cols = D(geoid=CAT, level=CAT, kind=CAT, npois=I32)
# pois = pois.astype(cols)[list(cols)].disp()
# pois.to_parquet('data/pois.parquet') # 21s
pois = pd.read_parquet('data/pois.parquet').disp()

437,955 rows x 4 cols; Memory: 17.4 MiB


,geoid,level,kind,npois
,<category>,<category>,<category>,<int32>
0,040130101022,BG,All,6


In [8]:
filt(pois, level='BG').groupby('kind', observed=0).npois.sum().to_frame().T

kind,All,Education,Groceries,Medical,Social Support
npois,2534020,94093,64488,326323,70785


## Combine job and POI counts

In [9]:
# opport = pd.concat([
#     jobs.assign(kind='Jobs: ' + jobs.kind.astype(str)).rename(columns=D(njobs='opport')),
#     pois.assign(kind='POIs: ' + pois.kind.astype(str)).rename(columns=D(npois='opport'))
# ]).reset_index(drop=1).astype(D(geoid=CAT, kind=CAT)).disp()
# opport.to_parquet('data/opportunities.parquet')
opport = pd.read_parquet('data/opportunities.parquet').disp()

1,595,036 rows x 4 cols; Memory: 42.5 MiB


,geoid,level,kind,opport
,<category>,<category>,<category>,<int32>
0,040019426001,BG,Jobs: All,4


## City summary statistics

In [10]:
bgs = (
    pd.read_parquet('data/zones.parquet', filters=[('level', '==', 'BG')])
    .merge(filt(jobs, kind='All', level='BG'), 'left')
    .merge(filt(pois, kind='All', level='BG'), 'left')
    .astype(D(county=str, ua_code=str, city=str))
    .drop(columns='geometry')
).disp()

106,044 rows x 11 cols; Memory: 29.8 MiB


,geoid,level,county,ua_code,city,aland,awater,popu,labor,njobs,npois
,<object>,<category>,<object>,<object>,<object>,<float32>,<float32>,<int32>,<int32>,<float64>,<float64>
0,280330705233,BG,DeSoto,56116,Memphis,1.008998,0.0,3924,1832,NaN,59.0


In [11]:
df = bgs.groupby(['ua_code', 'city']).agg(D(
    geoid='count', county=set, aland='sum', awater='sum',
    popu='sum', njobs='sum', npois='sum'
)).reset_index().rename(columns=D(geoid='nBG'))
ntract = (bgs.assign(geoid=bgs.geoid.str[:11])
          .groupby(['city', 'geoid']).size().reset_index()
          .groupby('city').geoid.count().rename('nTract').reset_index())
df = df.merge(ntract, on='city')
df['popdens'] = df.popu / df.aland
df['nCounty'] = df.county.apply(len)
df.index = df.popu.rank(ascending=0).astype(I16).rename('rank')
for col in ['popu', 'njobs', 'npois']:
    df[col] /= 1000
df = df['city ua_code nBG nTract nCounty popu aland popdens njobs npois'.split()]
df = df.sort_index().disp(None)
df.to_csv('data/city_summary_stats.csv')

50 rows x 10 cols; Memory: 0.0 MiB


,city,ua_code,nBG,nTract,nCounty,popu,aland,popdens,njobs,npois
rank,<object>,<object>,<int64>,<int64>,<int64>,<float64>,<float32>,<float64>,<float64>,<float64>
1,New York,63217,14305,4790,24,18718.361,4442.073242,4213.879416,8526.579,302.456
2,Los Angeles,51445,8085,2912,4,12329.329,2039.094849,6046.471555,5700.028,228.959
3,Chicago,16264,6276,2143,10,8609.691,3252.7771,2646.873959,4083.123,137.677
4,Miami,56602,3906,1478,4,6084.318,2790.271484,2180.546959,2546.562,111.681
5,Houston,40429,3447,1354,7,5847.375,2657.120605,2200.643429,2646.178,86.396
6,Dallas,22042,3446,1350,10,5742.504,2489.686279,2306.517109,3176.505,89.876
7,Philadelphia,69076,4328,1471,12,5724.087,2798.585938,2045.349733,2677.4,90.709
8,Washington,92242,3319,1231,13,5149.414,1742.771729,2954.726609,2591.127,78.248
9,Atlanta,03817,3353,1299,18,5021.762,3172.847412,1582.730383,2510.418,84.075


In [18]:
df = pd.read_csv('data/export/las-vegas/las-vegas_bg.csv').disp()

1,268 rows x 679 cols; Memory: 6.6 MiB


,Unnamed: 0,geoid,lon,lat,Co_jobEdu_Bi_15,Co_jobEdu_Bi_30,Co_jobEdu_Bi_45,Co_jobEdu_Bi_60,Co_jobEdu_Bi_90,Co_jobEdu_Dr_15,...,XM_poiTot_Dr_15,XM_poiTot_Dr_30,XM_poiTot_Dr_45,XM_poiTot_Dr_60,XM_poiTot_Dr_90,XM_poiTot_Wa_15,XM_poiTot_Wa_30,XM_poiTot_Wa_45,XM_poiTot_Wa_60,XM_poiTot_Wa_90
,<int64>,<int64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,...,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>
0,0,320030032492,-115.3386,36.18706,3116.0,10315.0,20495.0,38552.0,122673.0,39538.0,...,0.011903,0.010381,0.010196,0.010206,0.010205,0.001693,0.0012,0.001543,0.001545,0.000741
